# Image metrics generator
This notebook uses the images folders generated using [PIPELINE 1] and produces the image metric values for each image.

## Usage:
Run all cells

## Requirements:
-Image folders:\
    ./[task_type]/content/output_plots/[domain_type]/[domain_type]

## Outputs:
    ./[task_type]/content/output_plots/[domain_type]/[domain_type]_metrics_results

In [ ]:
import os
import image_metrics_utils
import json

### Calculate image metric scores

In [2]:
def calculate_scores(type,model_type,source_name):

    folder1_path = './'+type+'/content/output_plots/real/real/'
    folder2_path = './'+type+'/content/output_plots/'+model_type+'/'+str(source_name)+'/'
    os.makedirs('./'+type+'/content/output_plots/'+model_type+'/'+str(source_name)+'_metrics_results/', exist_ok=True)
    output_folder = './'+type+'/content/output_plots/'+model_type+'/'+str(source_name)+'_metrics_results/'
    
    # Load images from folders
    images1,_ = image_metrics_utils.load_images_from_folder(folder1_path)
    images2,path2 = image_metrics_utils.load_images_from_folder(folder2_path)
    
    # Calculate metrics
    results = []
    results_dict = {}


    print(type,model_type,source_name)
    
    print("Calculating SSIM scores...")
    ssim_scores = [image_metrics_utils.calculate_ssim(img1, img2) for img1, img2 in zip(images1, images2)]
    results.append(ssim_scores)
    results_dict["ssim_scores"]=ssim_scores
    
    print("Calculating PSNR scores...")
    psnr_scores = [image_metrics_utils.calculate_psnr(img1, img2) for img1, img2 in zip(images1, images2)]
    results.append(psnr_scores)
    results_dict["psnr_scores"]=psnr_scores
    
    print("Calculating MSE scores...")
    mse_scores = [image_metrics_utils.calculate_mse(img1, img2) for img1, img2 in zip(images1, images2)]
    results.append(mse_scores)
    results_dict["mse_scores"]=mse_scores
    
    print("Calculating cosine similarities...")
    cosine_similarities = image_metrics_utils.calculate_cosine_similarity(images1, images2)
    results.append(cosine_similarities)
    results_dict["cosine_similarities"]=cosine_similarities
    
    print("Calculating correlation coefficients...")
    corr_coeffs = [image_metrics_utils.calculate_correlation_coefficient(img1, img2) for img1, img2 in zip(images1, images2)]
    results.append(corr_coeffs)
    results_dict["corr_coeffs"]=corr_coeffs
    
    print("Calculating Texture Similarity...")
    text_sim=image_metrics_utils.calculate_texture_similarity(images1, images2)
    results.append(text_sim)
    results_dict["text_sim"]=text_sim
    
    print("Calculating WD Score...")
    wd_score=image_metrics_utils.calculate_wd(images1, images2)
    results.append(wd_score)
    results_dict["wd_score"]=wd_score
    
    print("Calculating KL Divergence...")
    kl_div=image_metrics_utils.calculate_kl_divergence(images1, images2)
    results.append(kl_div)
    results_dict["kl_div"]=kl_div
    
    print("Calculating Histogram Intersection...")
    hist_inter=image_metrics_utils.calculate_histogram_intersection(images1, images2)
    results.append(hist_inter)
    results_dict["hist_inter"]=hist_inter
    
    print("Calculating Perceptual Dist...")
    perc_dist= image_metrics_utils.calculate_perceptual_distances(images1, images2)
    results.append(perc_dist)
    results_dict["perc_dist"]=perc_dist

    print("Calculating Inception Score...")
    inception_score = image_metrics_utils.calculate_inception_score(images1,images2)
    results_dict["inception_score"] = inception_score
    print(results_dict["inception_score"])
    
    
    print("Calculating FID...")
    fid_score = image_metrics_utils.calculate_fid(images1, images2)
    results_dict["fid_score"] = fid_score

    print("Calculating Kernel Density Estimation (KDE)...")
    kde_scores, kde_probs = image_metrics_utils.calculate_kde(images2)
    results.append(kde_scores)
    results_dict["kde_scores"] = kde_scores
    results.append(kde_probs)
    results_dict["kde_probs"] = kde_probs

    if type=='donkey':
        print("Calculating Kernel Inception Distance (KID)...")
        kid_score = image_metrics_utils.calculate_kid_donkey(images1, images2)
        results_dict["kid_score"] = kid_score.numpy()
    else:
        print("Calculating Kernel Inception Distance (KID)...")
        kid_score = image_metrics_utils.calculate_kid_kitti(images1, images2)
        results_dict["kid_score"] = kid_score.numpy()
    

    print("Calculating Semantic seg score (SSS)...")
    sss = image_metrics_utils.calculate_semantic_segmentation_score(images1, images2)
    results_dict["ss_score"]=sss
    
    
    print("Saving jsons for "+source_name)
    for i in range(0,len(path2)):
        single_results_dict = {}
        for metric_name in results_dict:
            if (metric_name == metric_name == 'inception_score' or metric_name=="kid_score" or metric_name=="fid_score"):
                # print(metric_name)
                single_results_dict[metric_name] = float(results_dict[metric_name])
            else:
                # print(metric_name)
                single_results_dict[metric_name] = float(results_dict[metric_name][i])
        output_name=output_folder+path2[i].split("/")[-1].split("_")[0]+"_"+path2[i].split("/")[-1].split("_")[1].split(".")[0]+".json"
        with open(output_name, "w") as json_file:
            json.dump(single_results_dict, json_file)
    
    print("Results saved as JSON.")
    return results_dict



### Calculate image metrics for kitti domain type

In [ ]:
task_type='kitti'
model_type,source_name='sim','sim'
calculate_scores(task_type,model_type,source_name)
model_type='cyclegan'
source_names=['cyclegan_1','cyclegan_2','cyclegan_3']
for source_name in source_names:
    calculate_scores(task_type,model_type,source_name)
model_type='pix2pix_mask_manual'
source_names=['pix2pix_mask_1_sim','pix2pix_mask_2_sim','pix2pix_mask_3_sim']
for source_name in source_names:
    calculate_scores(task_type,model_type,source_name)

### Calculate image metrics for donkey domain type

In [ ]:
task_type='donkey'
model_type,source_name='sim','sim'
calculate_scores(task_type,model_type,source_name)
model_type='cyclegan'
source_names=['cyclegan_1','cyclegan_2','cyclegan_3']
for source_name in source_names:
    calculate_scores(task_type,model_type,source_name)
model_type='pix2pix_mask_manual'
source_names=['pix2pix_mask_1_sim','pix2pix_mask_2_sim','pix2pix_mask_3_sim']
for source_name in source_names:
    calculate_scores(task_type,model_type,source_name)